In [1]:
from joblib import dump, load
from pyproj import Transformer
import geopandas
import geopy
from geopy.geocoders import Nominatim
import folium
from geopy.extra.rate_limiter import RateLimiter
from folium import plugins
from folium.plugins import MarkerCluster
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import numpy as np
from shapely.geometry import Polygon
import pyproj
from os import listdir
from os.path import isfile, join

x1,y1 = 6584000,540500
transformer = Transformer.from_crs("epsg:3301", "epsg:4326")# from lambert est97 coords nickname to wsg84 nickname

transformer.transform(x1,y1)

df = pd.read_csv("../../data/locations_in_use.csv")
df = df.apply(pd.to_numeric, errors='ignore', downcast='integer')


In the following cell the inputs can be changed

In [2]:
kohad = ['KOHT_APTEEK', 'KOHT_AVALIK_KOHT', 'KOHT_BUROO_KONTOR', 'KOHT_BUSSIJAAM_BUSSITERMINAAL', 'KOHT_BUSSIPEATUS', 'KOHT_EHITUS', 'KOHT_HOTELL_MOTELL', 'KOHT_KASIINO', 'KOHT_KAUBAMAJA', 'KOHT_KAUPLUS', 'KOHT_KIOSK', 'KOHT_KULTUURIASUTUS', 'KOHT_LASTEASUTUS_TERRITOORIUM', 'KOHT_MUU KOHT', 'KOHT_MUU RUUM', 'KOHT_OOKLUBI_DISKOTEEK', 'KOHT_OPPEASUTUS_SELLE_TERRITOORIUM', 'KOHT_OUEALATEE', 'KOHT_OUEALA_LAHTINE_HOOV', 'KOHT_PANGAAUTOMAAT', 'KOHT_PARK', 'KOHT_PARKLA', 'KOHT_RAND', 'KOHT_RAUDTEEJAAM', 'KOHT_RAVIASUTUS', 'KOHT_REISILAEV', 'KOHT_RIIDEHOID', 'KOHT_SADAM', 'KOHT_SOOGIKOHT', 'KOHT_SPORDIRAJATIS', 'KOHT_TAKSO', 'KOHT_TANAV_VALJAK', 'KOHT_TANKLA', 'KOHT_TEENINDUSETTEVOTE', 'KOHT_TREPIKODA_KORIDOR', 'KOHT_TURG', 'KOHT_UHISSOIDUK']
for k in kohad:
    df[k] = 0
df.KOHT_AVALIK_KOHT = 1
#df.KOHT_BUSSIJAAM_BUSSITERMINAAL = 1
#df.KOHT_BUSSIPEATUS = 1
df.KOHT_PARK = 1
df.KOHT_PARKLA = 1
df.KOHT_TANAV_VALJAK = 1
df["ToimNadalapaev_Esmaspäev"] = 0
df["ToimNadalapaev_Kolmapäev"] = 0
df["ToimNadalapaev_Laupäev  "] = 0
df["ToimNadalapaev_Neljapäev"] = 1
df["ToimNadalapaev_Pühapäev"] = 0
df["ToimNadalapaev_Reede     "] = 0
df["ToimNadalapaev_Teisipäev"] = 0
df["Aasta"] = 2021
df["Kuu"] = 11
df["Päev"] = 4
df["Tund"] = 20
df.head()
y_valid = pd.read_csv("../../data/y_valid.csv")
y_valid = y_valid.apply(pd.to_numeric, errors='ignore', downcast='integer')
y_valid.drop(columns=["Kahjusumma", "Koht"], inplace=True)
y_valid.head()


,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS
0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [3]:
# loading models

onlyfiles = [f for f in listdir("../cleaner_models/trained_models/xgbClassifier") if isfile(join("../cleaner_models/trained_models/xgbClassifier", f))]

models_dict = dict()
for file in onlyfiles:
    file_tot_path = join("../cleaner_models/trained_models/xgbClassifier", file)
    print(file.split("_", 1)[1].split(".")[0])
    if not "LIIK_KONFLIKTOMAVOLI" in file:
        models_dict[file.split("_", 1)[1].split(".")[0]] = load(file_tot_path)
    else:
        models_dict["LIIK_KONFLIKT/OMAVOLI"] = load(file_tot_path)
print("loaded")
models = list()
for col in y_valid.columns:
    models.append(models_dict[col])

LIIK_AVALIKU_KORRA_RIKKUMINE
LIIK_JALGRATTA_MOPEEDI_VARGUS
LIIK_KEHALINE_VAARKOHTLEMINE
LIIK_KELMUS
LIIK_KONFLIKTOMAVOLI
LIIK_KOOLIVAGIVALD
LIIK_LIIKLUSRIKKUMINE
LIIK_MOOTORSOIDUKI_VARGUS
LIIK_MUU
LIIK_PISIVARGUS
LIIK_ROOVIMINE
LIIK_TULEKAHJU
LIIK_VANDALISM
LIIK_VARGUS
loaded


In [4]:
preds = list()
for col, model in list(zip(y_valid.columns, models)):
    preds.append(model.predict(df))

In [5]:
#keep only the most probable crime
def chooseBiggestProb(df):
    a = df.copy()
    for index, row in a.iterrows():
        maxindeks = row.idxmax()

        for i, item in row.iteritems():
            if i == maxindeks:
                row[i] = row[i]
            else:
                row[i] = 0
    a.head()
    return a
df_proba = pd.DataFrame(np.array(preds).transpose(), columns=y_valid.columns) # @rasmus ma ei oska kirjelda miks see rida vajalik aga igastahes see tuleb sellest probs_rfc kujust mis ei sobinud
df_proba = chooseBiggestProb(df_proba)
df_proba["Lest_Y"] = df.Lest_Y
df_proba["Lest_X"] = df.Lest_X
df_proba.head()

,LIIK_AVALIKU_KORRA_RIKKUMINE,LIIK_JALGRATTA_MOPEEDI_VARGUS,LIIK_KEHALINE_VAARKOHTLEMINE,LIIK_KELMUS,LIIK_KONFLIKT/OMAVOLI,LIIK_KOOLIVAGIVALD,LIIK_LIIKLUSRIKKUMINE,LIIK_MOOTORSOIDUKI_VARGUS,LIIK_MUU,LIIK_PISIVARGUS,LIIK_ROOVIMINE,LIIK_TULEKAHJU,LIIK_VANDALISM,LIIK_VARGUS,Lest_Y,Lest_X
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,542000,6588500
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,529000,6476000
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,543500,6588500
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,547000,6589000
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,592500,6470500


In [6]:
# instead of normal dataframe use geodf
data = geopandas.GeoDataFrame()


data["geoid"] = df_proba.index.astype(str)
for col in df_proba.columns:
    if not "Lest" in col:
        data[col] = df_proba[col]

# squares from original data
topleft = list(zip(df_proba["Lest_Y"], df_proba["Lest_X"]))
topright = list(zip(df_proba["Lest_Y"], df_proba["Lest_X"]+500))
bottomleft = list(zip(df_proba["Lest_Y"]+500, df_proba["Lest_X"]))
bottomright = list(zip(df_proba["Lest_Y"]+500, df_proba["Lest_X"]+500))
temp = geopandas.GeoSeries([Polygon([a, b, c, d]) for a,b,c, d in list(zip(topleft, topright, bottomright, bottomleft))])

data["geometry"] = temp

print(data.info())

#data from lambert
data.crs = pyproj.CRS.from_epsg(3301)
# Re-project to WGS84
data = data.to_crs(epsg=4326)

data.iloc[0]


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2296 entries, 0 to 2295
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   geoid                          2296 non-null   object  
 1   LIIK_AVALIKU_KORRA_RIKKUMINE   2296 non-null   int8    
 2   LIIK_JALGRATTA_MOPEEDI_VARGUS  2296 non-null   int8    
 3   LIIK_KEHALINE_VAARKOHTLEMINE   2296 non-null   int8    
 4   LIIK_KELMUS                    2296 non-null   int8    
 5   LIIK_KONFLIKT/OMAVOLI          2296 non-null   int8    
 6   LIIK_KOOLIVAGIVALD             2296 non-null   int8    
 7   LIIK_LIIKLUSRIKKUMINE          2296 non-null   int8    
 8   LIIK_MOOTORSOIDUKI_VARGUS      2296 non-null   int8    
 9   LIIK_MUU                       2296 non-null   int8    
 10  LIIK_PISIVARGUS                2296 non-null   int8    
 11  LIIK_ROOVIMINE                 2296 non-null   int8    
 12  LIIK_TULEKAHJU            

geoid                                                                            0
LIIK_AVALIKU_KORRA_RIKKUMINE                                                     0
LIIK_JALGRATTA_MOPEEDI_VARGUS                                                    0
LIIK_KEHALINE_VAARKOHTLEMINE                                                     0
LIIK_KELMUS                                                                      0
LIIK_KONFLIKT/OMAVOLI                                                            0
LIIK_KOOLIVAGIVALD                                                               0
LIIK_LIIKLUSRIKKUMINE                                                            0
LIIK_MOOTORSOIDUKI_VARGUS                                                        0
LIIK_MUU                                                                         0
LIIK_PISIVARGUS                                                                  0
LIIK_ROOVIMINE                                                                   0
LIIK

In [12]:
# multilayered map of chloroform
# maagia et saada kujule Polygon(alguslat alguslong, lopplat lopplong)
# TODO vb see custom tule siin læheb katki sest free tier, teha siis uus konto ja v¨tta uus token
# https://www.jawg.io/lab/styles
map2 = folium.Map([58.87, 25.49], zoom_start=7,
                  tiles="https://{s}.tile.jawg.io/jawg-light/{z}/{x}/{y}{r}.png?access-token=QWqVx55clvOrvgKqv3BFyTjV4K3qJj9eiyrIT75TnuhC5WLOk4bkEcHs2RKc7Ubd",
                  attr='<a href="http://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

tempdata = data[data.columns].replace(['0', 0], np.nan)
cnter = -1
colorList = ["Blues", "Reds", "Greens", "Greys", "Purples", "Oranges"]
for col in df_proba:
    if not "Lest_" in col and sum(df_proba[col]) > 0:
        cnter+=1
        folium.Choropleth(
            geo_data=tempdata,
            name=col,
            data=tempdata,
                columns=['geoid', col],
            key_on='feature.id',
            fill_color=colorList[cnter%len(colorList)],
            fill_opacity=0.5,
            line_opacity=0.2,
            line_color='white',
            line_weight=0,
            nan_fill_color="blue",
            nan_fill_opacity=0.0,
            highlight=False,
            smooth_factor=0.5,
            #threshold_scale=[100, 250, 500, 1000, 2000],/
            legend_name= col).add_to(map2)

folium.LayerControl(autoZIndex=False, collapsed=True).add_to(map2) # annab midagi et kuidagi layer toggelida
map2.save("chloroform6.html")
#map2

In [8]:
for i in df_proba.columns:
    print(sum(df_proba[i]), i)

563 LIIK_AVALIKU_KORRA_RIKKUMINE
12 LIIK_JALGRATTA_MOPEEDI_VARGUS
15 LIIK_KEHALINE_VAARKOHTLEMINE
0 LIIK_KELMUS
0 LIIK_KONFLIKT/OMAVOLI
0 LIIK_KOOLIVAGIVALD
0 LIIK_LIIKLUSRIKKUMINE
0 LIIK_MOOTORSOIDUKI_VARGUS
352 LIIK_MUU
0 LIIK_PISIVARGUS
0 LIIK_ROOVIMINE
2 LIIK_TULEKAHJU
45 LIIK_VANDALISM
191 LIIK_VARGUS
1341964500 Lest_Y
15012198500 Lest_X


In [9]:
for i in df_proba.columns:
    print(sum(df_proba[i]), i)

563 LIIK_AVALIKU_KORRA_RIKKUMINE
12 LIIK_JALGRATTA_MOPEEDI_VARGUS
15 LIIK_KEHALINE_VAARKOHTLEMINE
0 LIIK_KELMUS
0 LIIK_KONFLIKT/OMAVOLI
0 LIIK_KOOLIVAGIVALD
0 LIIK_LIIKLUSRIKKUMINE
0 LIIK_MOOTORSOIDUKI_VARGUS
352 LIIK_MUU
0 LIIK_PISIVARGUS
0 LIIK_ROOVIMINE
2 LIIK_TULEKAHJU
45 LIIK_VANDALISM
191 LIIK_VARGUS
1341964500 Lest_Y
15012198500 Lest_X


In [10]:
for i in df_proba.columns:
    print(sum(df_proba[i]), i)

563 LIIK_AVALIKU_KORRA_RIKKUMINE
12 LIIK_JALGRATTA_MOPEEDI_VARGUS
15 LIIK_KEHALINE_VAARKOHTLEMINE
0 LIIK_KELMUS
0 LIIK_KONFLIKT/OMAVOLI
0 LIIK_KOOLIVAGIVALD
0 LIIK_LIIKLUSRIKKUMINE
0 LIIK_MOOTORSOIDUKI_VARGUS
352 LIIK_MUU
0 LIIK_PISIVARGUS
0 LIIK_ROOVIMINE
2 LIIK_TULEKAHJU
45 LIIK_VANDALISM
191 LIIK_VARGUS
1341964500 Lest_Y
15012198500 Lest_X
